In [4]:
import numpy as np
import cv2
import os

img_resize = 60

folders = ['triangle', 'star', 'square', 'circle']
shapes = []
images = []
for folder in folders:
    for path in os.listdir('shapes/'+folder):
        img = cv2.imread('shapes/'+folder+'/'+path,0)
        images.append(cv2.resize(img, (img_resize, img_resize)))
        shapes.append(folders.index(folder))
    
to_train= 0
train_images, test_images, train_shapes, test_shapes = [],[],[],[]
for image, shape in zip(images, shapes):
    if to_train<5:
        train_images.append(image)
        train_shapes.append(shape)
        to_train+=1
    else:
        test_images.append(image)
        test_shapes.append(shape)
        to_train = 0

def flatten(data_d, img):
    img = np.array(img)
    img = img.reshape(len(img), data_d)
    img = img.astype('float32')
    img /=255
    return img


from keras.utils import to_categorical

data_d = np.prod(images[0].shape)
train_data  = flatten(data_d, train_images)
test_data = flatten(data_d, test_images)


train_shapes = np.array(train_shapes)
test_shapes = np.array(test_shapes)
train_shapes_1 = to_categorical(train_shapes)
test_shapes_1 = to_categorical(test_shapes)


classes = np.unique(train_shapes)
class_n  = len(classes)

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout


cnn = Sequential()
cnn.add(Dense(256, activation = 'tanh', input_shape = (data_d,)))
cnn.add(Dropout(0.2))
cnn.add(Dense(256, activation='tanh'))
cnn.add(Dropout(0.2))
cnn.add(Dense(256, activation='relu'))
cnn.add(Dropout(0.2))
cnn.add(Dense(class_n, activation='softmax'))

cnn.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
history = cnn.fit(train_data, train_shapes_1, batch_size = 256, epochs=10, verbose=1,
                    validation_data=(test_data, test_shapes_1))

Train on 12475 samples, validate on 2495 samples
Epoch 1/10
12475/12475 [==============================] - 2s 127us/step - loss: 1.4748 - acc: 0.2468 - val_loss: 1.3798 - val_acc: 0.2517
Epoch 2/10
12475/12475 [==============================] - 1s 100us/step - loss: 1.2939 - acc: 0.3303 - val_loss: 1.2150 - val_acc: 0.2485
Epoch 3/10
12475/12475 [==============================] - 1s 99us/step - loss: 0.8560 - acc: 0.5135 - val_loss: 0.7252 - val_acc: 0.5002
Epoch 4/10
12475/12475 [==============================] - 1s 99us/step - loss: 0.7583 - acc: 0.5944 - val_loss: 0.5068 - val_acc: 0.7078
Epoch 5/10
12475/12475 [==============================] - 1s 100us/step - loss: 0.6783 - acc: 0.7007 - val_loss: 0.4022 - val_acc: 0.8826
Epoch 6/10
12475/12475 [==============================] - 1s 110us/step - loss: 0.5008 - acc: 0.8202 - val_loss: 0.2198 - val_acc: 0.9242
Epoch 7/10
12475/12475 [==============================] - 1s 116us/step - loss: 0.4662 - acc: 0.8341 - val_loss: 0.0962 - val

In [3]:
size = images[0].shape
size

(60, 60)

In [4]:
img = images[0]
img_new = np.ones([size[0],size[1],3])*255
black = np.argwhere(img == 0) 
for i in black:
    img_new[i[0]][i[1]][2] = 255
    img_new[i[0]][i[1]][0] = 0
    img_new[i[0]][i[1]][1] = 0
    

cv2.imwrite('color.jpg',img_new)

True

In [5]:
pred = model.predict(test_data)

In [6]:
pred

array([[  9.91542459e-01,   3.11455200e-03,   4.82049072e-03,
          5.22468006e-04],
       [  9.92202520e-01,   1.23445969e-03,   5.97854471e-03,
          5.84440248e-04],
       [  9.92729604e-01,   1.16662134e-03,   5.55847865e-03,
          5.45273535e-04],
       ..., 
       [  6.50335778e-07,   5.62466482e-08,   3.55572859e-03,
          9.96443570e-01],
       [  6.49323113e-07,   5.61942599e-08,   3.54338787e-03,
          9.96455848e-01],
       [  7.26018015e-07,   6.30694501e-08,   4.28182632e-03,
          9.95717347e-01]], dtype=float32)

In [7]:
import random 
import _pickle as pickles

In [8]:
class colour_env:
    def __init__(self, fn):
        
        self.img = fn
        self.img_new = np.ones([size[0],size[1],3])*255

        
        self._actions = ['bp','bn','gp','gn','rp','rn']
        self.coordinates = [0,0,0]
        
    def color_img(self):
        self.img_new = np.ones([size[0],size[1],3])*255
        black = np.argwhere(self.img == 0) 
        for i in black:
            self.img_new[i[0]][i[1]][0] = self.coordinates[0]
            self.img_new[i[0]][i[1]][1] = self.coordinates[1]
            self.img_new[i[0]][i[1]][2] = self.coordinates[2]
        cv2.imwrite('color_test.jpg',self.img_new)
        

    def get_cur_state(self):
        self.check_state()
        return self.coordinates
    
    def init(self):
        return self.coordinates
    
    def do_action(self,action):
        if action == 'bp':
            self.coordinates[0] += 65
            self.coordinates[1] -= 65
            self.coordinates[2] -= 65
        elif action == 'bn':
            self.coordinates[0] -= 65
            self.coordinates[1] += 65
            self.coordinates[2] += 65
        elif action == 'gp':
            self.coordinates[1] += 65
            self.coordinates[0] -= 65
            self.coordinates[2] -= 65
        elif action == 'gn':
            self.coordinates[1] -= 65
            self.coordinates[0] += 65
            self.coordinates[2] += 65
        elif action == 'rp':
            self.coordinates[2] += 65
            self.coordinates[0] -= 65
            self.coordinates[1] -= 65
        elif action == 'rn':
            self.coordinates[2] -=65
            self.coordinates[1] += 65
            self.coordinates[0] += 65
            
    def check_state(self):
        if self.coordinates[0] > 255:
            self.coordinates[0] = 255
        if self.coordinates[1] > 255:
            self.coordinates[1] = 255
        if self.coordinates[2] > 255:
            self.coordinates[2] = 255
        if self.coordinates[0] < 0:
            self.coordinates[0] = 0
        if self.coordinates[1] < 0:
            self.coordinates[1] = 0
        if self.coordinates[2] < 0:
            self.coordinates[2] = 0
            
#         print('in check state',self.coordinates)
            
        
    def reward(self, a, shape):
        self.color_img()
        if shape == 0:
            if np.argmax(self.coordinates) == 0:
                if self.coordinates[0] == 255 and self.coordinates[1] == 0 and self.coordinates[2] == 0:
                    return 30
                return 10
            elif self.coordinates[0] == 0:
                return -30
            else:
                return -10
        
        if shape == 1:
            if self.coordinates[0] == 127 and self.coordinates[1] == 0 and self.coordinates[2] == 127:
                return 30
            if np.fabs(self.coordinates[0]-127) < 10 and np.fabs(self.coordinates[2]-127) < 10 and np.fabs(self.coordinates[1]-0) < 10 :
                return 10
            if np.fabs(self.coordinates[0]-127) < 50 and np.fabs(self.coordinates[2]-127) < 50 and np.fabs(self.coordinates[1]-0) < 50 :
                return -30
            else:
                return -10
            
        if shape == 2:
            if np.argmax(self.coordinates) == 2:
                if self.coordinates[2] == 255 and self.coordinates[1] == 0 and self.coordinates[0] == 0:
                    return 30
                return 10
            elif self.coordinates[2] == 0:
                return -30
            else:
                return -10
        
        if shape == 3:
            if np.argmax(self.coordinates) == 1:
                if self.coordinates[1] == 255 and self.coordinates[0] == 0 and self.coordinates[2] == 0:
                    return 30
                return 10
            elif self.coordinates[1] == 0:
                return -30
            else:
                return -10
        
        
    def is_goal(self, r):
        if r == 30:
            return True
        else:
            return False
        
    def get_actions(self):
        return self._actions

In [9]:
class RLAgent: 
    def __init__(self, env,shape):
        self.shape = shape
        self.env = env
        self.n_a = len(env.get_actions())
        self.Q = {}
        
    def check_s(self,s):
        if s[0] > 255:
            s[0] = 255
        if s[1] > 255:
            s[1] = 255
        if s[2] > 255:
            s[2] = 255
        if s[0] < 0:
            s[0] = 0
        if s[1] < 0:
            s[1] = 0
        if s[2] < 0:
            s[2] = 0
        return s
    
    def reading_shape(self):
        if self.shape == 0:
            with open('file0.txt','rb') as handle:
                self.Q = pickles.loads(handle.read())
        if self.shape == 1:
            with open('file1.txt','rb') as handle:
                self.Q = pickles.loads(handle.read())
        if self.shape == 2:
            with open('file2.txt','rb') as handle:
                self.Q = pickles.loads(handle.read())
        if self.shape == 3:
            with open('file3.txt','rb') as handle:
                self.Q = pickles.loads(handle.read())
                
    def writing_shape(self):
        if self.shape == 0:
            with open ('file0.txt','wb') as handle:
                pickles.dump(self.Q,handle)
        if self.shape == 1:
            with open ('file1.txt','wb') as handle:
                pickles.dump(self.Q,handle)
        if self.shape == 2:
            with open ('file2.txt','wb') as handle:
                pickles.dump(self.Q,handle)
        if self.shape == 3:
            with open ('file3.txt','wb') as handle:
                pickles.dump(self.Q,handle)
    
    def epsilon_greed(self, epsilon, s):
        s = self.check_s(s)
        s = tuple(s)
        print(s)
        if s not in self.Q:
            self.Q[s] = dict((action, 0) for action in env.get_actions())
        
        if np.random.rand() < epsilon:
            return random.choice(env.get_actions())
        else:
            Q_maximum = max(self.Q[s].values())
            possible_actions = []
            
            for i in list(env.get_actions()):
                if(self.Q[s][i] == Q_maximum):
                    possible_actions.append(i)
            return random.choice(possible_actions)  

    def train(self, **params):
        
        # parameters
        gamma = params.pop('gamma', 0.99)
        alpha = params.pop('alpha', 0.1)
        epsilon= params.pop('epsilon', 0.1)
        maxiter= params.pop('maxiter', 100) 
        maxstep= params.pop('maxstep', 1000) 
        
       
        s = self.env.init()
        
        for j in range(maxiter):
            s = self.env.get_cur_state()
            a = self.epsilon_greed(epsilon, s)
            s = tuple(s)
            rewards = []
            
            for step in range(maxstep):
                s1 = self.env.get_cur_state()
                a1 = self.epsilon_greed(epsilon,s1)
                s1 = tuple(s1)
                self.env.do_action(a1)
                
                reward = self.env.reward(a,self.shape)
                rewards.append(reward)
                self.Q[s][a] = self.Q[s][a] + alpha * (reward + (gamma*max(self.Q[s1].values()))- self.Q[s][a])
                
                if self.env.is_goal(reward):
                    self.Q[s1][a1] = 0
                    break
                    
                s = s1
                a = a1
        self.writing_shape()
        return rewards 
    
    def test(self, maxstep=1000):
        epsilon = 0.1
        self.reading_shape()
        s = self.env.get_cur_state()
        
        a = self.epsilon_greed(epsilon, s)
        s = tuple(s)
        rewards = []
        
        for step in range(maxstep):
                s1 = self.env.get_cur_state()
                
                a1 = self.epsilon_greed(epsilon,s1)
                s1 = tuple(s1)
                self.env.do_action(a1)
                
                reward = self.env.reward(a,self.shape)
                rewards.append(reward)
                print('step',step)
                if self.env.is_goal(reward):
                    self.Q[s1][a1] = 0
                    break
                    
                s = s1
                a = a1
        return rewards

In [18]:
img = cv2.imread('17.png',0)
def flatten(dimData, images):
    images = np.array(images)
    images = images.reshape(1, dimData)
    images = images.astype('float32')
    images /=255
    return images

img_new = cv2.resize(img, (60, 60))
dataDim = np.prod(img_new.shape)

cv2.imwrite('color_test.jpg',img_new)

True

In [19]:
testing = flatten(dataDim,img_new)

In [20]:
pred = model.predict(testing)
pred = np.argmax(pred)

In [21]:
pred

3

In [22]:
env = colour_env(img_new)
agent = RLAgent(env,pred)
rtrace = agent.train(gamma=0.99, 
                    alpha=0.1, 
                    epsilon=0.1, 
                    maxiter=100, 
                    maxstep=100)

(0, 0, 0)
(0, 0, 0)
(65, 0, 65)
(0, 65, 0)
(0, 130, 65)
(0, 195, 0)
(0, 255, 0)
(65, 255, 0)
(0, 255, 65)
(0, 255, 130)
(65, 255, 65)
(0, 190, 130)
(65, 125, 65)
(0, 60, 130)
(65, 125, 65)
(130, 60, 130)
(65, 125, 195)
(130, 60, 255)
(65, 125, 255)
(0, 190, 190)
(65, 125, 125)
(0, 60, 190)
(0, 125, 125)
(65, 60, 190)
(0, 125, 125)
(65, 60, 190)
(0, 125, 255)
(65, 190, 190)
(0, 255, 255)
(0, 255, 190)
(65, 190, 255)
(0, 255, 190)
(65, 190, 255)
(130, 125, 190)
(195, 60, 255)
(255, 0, 190)
(190, 65, 255)
(125, 130, 255)
(60, 195, 255)
(0, 255, 255)
(0, 255, 255)
(65, 190, 255)
(0, 255, 255)
(65, 190, 255)
(130, 125, 255)
(65, 60, 255)
(0, 125, 255)
(65, 190, 190)
(0, 255, 255)
(65, 190, 190)
(0, 255, 255)
(65, 190, 190)
(0, 255, 255)
(65, 190, 190)
(0, 255, 255)
(65, 190, 190)
(0, 255, 255)
(65, 190, 190)
(0, 255, 255)
(65, 190, 190)
(0, 255, 255)
(65, 190, 190)
(130, 125, 125)
(195, 60, 60)
(130, 125, 0)
(195, 60, 0)
(255, 125, 0)
(255, 60, 65)
(255, 0, 0)
(190, 0, 65)
(255, 0, 0)
(255,

(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(0, 190, 130)
(65, 255, 65)
(130, 190, 130)
(65, 255, 195)
(130, 190, 255)
(65, 255, 190)
(0, 255, 125)
(0, 255, 60)
(65, 190, 0)
(0, 255, 0)
(65, 255, 0)
(0, 255, 65)
(0, 255, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(130, 255, 0)
(65, 190, 65)
(0, 255, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(0, 255, 65)
(0, 255, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(130, 190, 0)
(195, 255, 0)
(130, 255, 0)
(65, 190, 65)
(0, 255, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190

(60, 255, 65)
(125, 190, 130)
(190, 255, 65)
(190, 255, 65)
(125, 255, 0)
(60, 255, 0)
(125, 190, 0)
(60, 255, 0)
(125, 190, 0)
(60, 255, 0)
(125, 190, 0)
(60, 255, 0)
(125, 190, 0)
(60, 255, 0)
(125, 190, 65)
(60, 255, 0)
(125, 190, 0)
(60, 255, 0)
(125, 190, 0)
(60, 255, 0)
(125, 190, 0)
(60, 255, 0)
(125, 190, 0)
(60, 255, 0)
(125, 190, 0)
(60, 255, 0)
(125, 190, 0)
(60, 255, 0)
(125, 190, 0)
(60, 255, 0)
(125, 190, 0)
(60, 255, 0)
(125, 190, 0)
(60, 255, 0)
(125, 190, 0)
(60, 125, 65)
(125, 60, 0)
(190, 0, 65)
(125, 65, 130)
(60, 130, 65)
(0, 195, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0,

(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(130, 190, 0)
(195, 255, 0)
(130, 255, 0)
(65, 190, 65)
(0, 255, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(0, 125, 195)
(65, 190, 130)
(130, 125, 65)
(65, 190, 0)
(0, 255, 0)
(65, 255, 0)
(0, 255, 65)
(0, 255, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 125, 65)
(0, 190, 0)
(65, 255, 0)
(0, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)

(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(130, 190, 0)
(195, 255, 0)
(130, 255, 0)
(65, 190, 65)
(0, 255, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 125, 195)
(130, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 125, 65)
(0, 190, 0)
(65, 255, 0)
(0, 255, 65)
(0, 255, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(0, 125, 195)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 255, 0)
(65, 255, 0)
(0, 255, 65)
(0, 255, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 13

...

(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 125, 65)
(0, 190, 0)
(65, 255, 0)
(0, 255, 65)
(0, 255, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 125, 195)
(130, 190, 130)
(65, 255, 65)
(0, 190, 130)
(0, 125, 195)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(130, 190, 130)
(65, 255, 65)
(130, 255, 0)
(65, 190, 65)
(0, 255, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(130, 190, 0)
(195, 255, 0)
(130, 255, 0)
(65, 190, 65)
(0, 255, 130)
(0, 255, 195)
(0, 255, 130)
(65, 255, 65)
(0, 1

(195, 255, 0)
(130, 255, 0)
(65, 190, 65)
(0, 255, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(130, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(0, 255, 65)
(0, 255, 130)
(65, 255, 65)
(0, 190, 130)
(65, 125, 65)
(0, 190, 0)
(65, 125, 65)
(0, 190, 0)
(65, 255, 0)
(0, 255, 65)
(0, 255, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 255, 0)
(65, 255, 0)
(0, 255, 65)
(65, 255, 0)
(0, 255, 65)
(0, 255, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 255, 0)
(65, 255, 0)
(0, 255, 65)
(0, 255, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 255, 130)
(65, 255, 65)
(0, 190, 130)
(65, 25

(0, 190, 130)
(65, 255, 65)
(130, 190, 0)
(195, 255, 0)
(130, 255, 0)
(65, 190, 65)
(0, 255, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(0, 125, 195)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(0, 255, 65)
(0, 255, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 255, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190

(0, 255, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(130, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 125, 195)
(130, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 255, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(

(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(130, 190, 0)
(195, 255, 0)
(130, 255, 0)
(195, 190, 65)
(255, 255, 0)
(190, 255, 65)
(125, 255, 0)
(60, 255, 0)
(125, 190, 0)
(60, 255, 0)
(125, 190, 0)
(60, 255, 0)
(125, 190, 0)
(60, 255, 0)
(125, 190, 0)
(60, 255, 0)
(125, 255, 0)
(60, 255, 0)
(125, 190, 0)
(125, 190, 0)
(60, 255, 0)
(125, 190, 0)
(60, 255, 0)
(125, 190, 0)
(60, 255, 0)
(0, 255, 65)
(65, 255, 0)
(0, 255, 65)
(0, 255, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(130, 255, 0)
(65, 190, 65)
(0, 255, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 255, 0)
(65, 255, 0)
(0, 255, 65)
(0, 255, 130)
(65, 255, 65)
(0, 190, 130)
(65, 255, 65)
(0, 190, 130)
(0, 255, 195)
(0, 255, 130)
(0, 255, 65)
(0, 255

In [23]:
agent.env.coordinates = [0,0,0]
agent.test(1000)

(0, 0, 0)
(0, 0, 0)
step 0
(65, 0, 65)
step 1
(0, 65, 0)
step 2
(0, 130, 65)
step 3
(0, 195, 0)
step 4
(0, 255, 0)
step 5
(65, 255, 0)
step 6
(0, 255, 65)
step 7
(0, 255, 130)
step 8
(65, 255, 65)
step 9
(0, 190, 130)
step 10
(65, 255, 65)
step 11
(0, 190, 130)
step 12
(65, 255, 65)
step 13
(0, 190, 130)
step 14
(65, 255, 65)
step 15
(0, 190, 130)
step 16
(65, 255, 65)
step 17
(0, 190, 130)
step 18
(65, 255, 65)
step 19
(0, 190, 130)
step 20
(65, 255, 65)
step 21
(0, 190, 130)
step 22
(65, 255, 65)
step 23
(0, 190, 130)
step 24
(65, 255, 65)
step 25
(0, 190, 130)
step 26
(65, 255, 65)
step 27
(0, 190, 130)
step 28
(65, 255, 65)
step 29
(0, 190, 130)
step 30
(65, 255, 65)
step 31
(0, 190, 130)
step 32
(65, 255, 65)
step 33
(0, 255, 0)
step 34
(65, 255, 0)
step 35
(0, 255, 65)
step 36
(0, 255, 130)
step 37
(65, 255, 65)
step 38
(0, 190, 130)
step 39
(0, 255, 195)
step 40
(0, 255, 130)
step 41
(65, 255, 65)
step 42
(0, 190, 130)
step 43
(65, 255, 65)
step 44
(0, 190, 130)
step 45
(65, 255

step 369
(65, 255, 65)
step 370
(0, 190, 130)
step 371
(65, 255, 65)
step 372
(0, 190, 130)
step 373
(65, 255, 65)
step 374
(0, 190, 130)
step 375
(65, 255, 65)
step 376
(0, 190, 130)
step 377
(65, 255, 65)
step 378
(0, 190, 130)
step 379
(65, 255, 65)
step 380
(0, 190, 130)
step 381
(65, 255, 65)
step 382
(0, 190, 130)
step 383
(65, 255, 65)
step 384
(0, 190, 130)
step 385
(0, 255, 195)
step 386
(0, 255, 130)
step 387
(65, 255, 65)
step 388
(0, 190, 130)
step 389
(0, 255, 195)
step 390
(0, 255, 130)
step 391
(65, 255, 65)
step 392
(0, 190, 130)
step 393
(65, 255, 65)
step 394
(0, 190, 130)
step 395
(65, 255, 65)
step 396
(0, 190, 130)
step 397
(65, 255, 65)
step 398
(0, 190, 130)
step 399
(65, 255, 65)
step 400
(0, 190, 130)
step 401
(65, 255, 65)
step 402
(0, 190, 130)
step 403
(65, 255, 65)
step 404
(0, 255, 130)
step 405
(0, 255, 65)
step 406
(0, 255, 130)
step 407
(65, 255, 65)
step 408
(0, 190, 130)
step 409
(0, 255, 195)
step 410
(65, 255, 130)
step 411
(0, 255, 65)
step 412
(0,

step 764
(0, 190, 130)
step 765
(65, 255, 65)
step 766
(0, 190, 130)
step 767
(65, 255, 65)
step 768
(0, 190, 130)
step 769
(65, 255, 65)
step 770
(0, 190, 130)
step 771
(65, 255, 65)
step 772
(0, 190, 130)
step 773
(65, 255, 65)
step 774
(0, 190, 130)
step 775
(65, 255, 65)
step 776
(0, 190, 130)
step 777
(65, 255, 65)
step 778
(0, 190, 130)
step 779
(65, 255, 65)
step 780
(0, 190, 130)
step 781
(65, 255, 65)
step 782
(0, 190, 130)
step 783
(65, 255, 65)
step 784
(130, 190, 0)
step 785
(195, 255, 0)
step 786
(130, 255, 0)
step 787
(65, 190, 65)
step 788
(0, 255, 130)
step 789
(65, 255, 65)
step 790
(0, 190, 130)
step 791
(65, 255, 65)
step 792
(0, 190, 130)
step 793
(65, 255, 65)
step 794
(0, 190, 130)
step 795
(0, 125, 195)
step 796
(0, 190, 130)
step 797
(65, 255, 65)
step 798
(0, 255, 0)
step 799
(0, 255, 65)
step 800
(0, 255, 130)
step 801
(65, 255, 65)
step 802
(0, 190, 130)
step 803
(65, 255, 65)
step 804
(130, 190, 130)
step 805
(65, 255, 65)
step 806
(0, 190, 130)
step 807
(65

[-10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 -10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10